In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import folium
import requests
from pathlib import Path
import os

In [2]:
# Configurations for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
plt.style.use('seaborn-v0_8-darkgrid')

In [3]:
# Function to load data from API
def load_data_from_api(url):
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)

# Loading data from APIs (if used)
# Example API for traffic stops
traffic_stops_url = 'https://opendata.maryland.gov/resource/tx73-47dk.json'
traffic_stops_data = load_data_from_api(traffic_stops_url)

In [4]:
traffic_stops_data.info()
traffic_stops_data.columns
traffic_stops_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   year                                  41 non-null     object
 1   quarter                               41 non-null     object
 2   quarter_and_year                      41 non-null     object
 3   medevac_missions                      41 non-null     object
 4   impaired_driving_arrests              41 non-null     object
 5   cross_border_criminal_investigations  41 non-null     object
 6   total_traffic_stops                   41 non-null     object
dtypes: object(7)
memory usage: 2.4+ KB


,year,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops
0,2014,Q1,Q1 2014,376,1496,180,110734
1,2014,Q2,Q2 2014,597,1630,104,141940
2,2014,Q3,Q3 2014,681,1767,148,147807
3,2014,Q4,Q4 2014,484,1562,108,119741
4,2015,Q1,Q1 2015,345,1500,210,132779


In [5]:
traffic_stops_data.tail()

,year,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops
36,2023,Q1,Q1 2023,371,1256,320,98042
37,2023,Q2,Q2 2023,543,1241,333,87591
38,2023,Q3,Q3 2023,548,1163,294,80525
39,2023,Q4,Q4 2023,437,1158,243,79072
40,2024,Q1,Q1 2024,329,1211,299,92633


In [6]:
# Define the base directory
base_dir = Path('projectdata')

# Specific files
aadt_points_file = base_dir / 'Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Points).csv'
aadt_lines_file = base_dir / 'Maryland_Annual_Average_Daily_Traffic_-_Annual_Average_Daily_Traffic_(SHA_Statewide_AADT_Lines).csv'
pedestrian_injury_file = base_dir / 'SHIP_Pedestrian_Injury_Rate_on_Public_Roads_2009-2022_20240505.csv'

In [7]:
if not aadt_points_file.exists():
    print(f"The file {aadt_points_file} does not exist!")
else:
    aadt_points_data = pd.read_csv(aadt_points_file)

if not aadt_lines_file.exists():
    print(f"The file {aadt_lines_file} does not exist!")
else:
    aadt_lines_data = pd.read_csv(aadt_lines_file)

if not pedestrian_injury_file.exists():
    print(f"The file {pedestrian_injury_file} does not exist!")
else:
    pedestrian_injury_data = pd.read_csv(pedestrian_injury_file)

C:\Users\madga\AppData\Local\Temp\ipykernel_38592\3432739259.py:4: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_points_data = pd.read_csv(aadt_points_file)
C:\Users\madga\AppData\Local\Temp\ipykernel_38592\3432739259.py:9: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_lines_data = pd.read_csv(aadt_lines_file)


In [8]:
aadt_points_data = pd.read_csv(aadt_points_file)
aadt_lines_data = pd.read_csv(aadt_lines_file)
pedestrian_injury_data = pd.read_csv(pedestrian_injury_file)

C:\Users\madga\AppData\Local\Temp\ipykernel_38592\3372450109.py:1: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_points_data = pd.read_csv(aadt_points_file)
C:\Users\madga\AppData\Local\Temp\ipykernel_38592\3372450109.py:2: DtypeWarning: Columns (98) have mixed types. Specify dtype option on import or set low_memory=False.
  aadt_lines_data = pd.read_csv(aadt_lines_file)


In [9]:
# General cleaning for all datasets
def clean_data(df):
    # Example: Convert dates, handle missing values
    df.dropna(inplace=True)
    return df

aadt_points_data = clean_data(aadt_points_data)
aadt_lines_data = clean_data(aadt_lines_data)
pedestrian_injury_data = clean_data(pedestrian_injury_data)

In [10]:
aadt_lines_data.info()
aadt_lines_data.columns
aadt_lines_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 187 entries, 46 to 4658
Columns: 102 entries, OBJECTID to Shape_Length
dtypes: float64(52), int64(11), object(39)
memory usage: 150.5+ KB


,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT_2010,AADT_2011,AADT_2012,AADT_2013,AADT_2014,AADT_2015,AADT_2016,AADT_2017,AADT_2018,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,Shape_Length0,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK_AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Shape_Length
46,47,B060073,6,Carroll,155,Westminster,E Green St,MU,210,,0.23,0.17,0.98,Green St E&w-between Washington Rd & Bishop St,Washington Rd To Md 27,Urban,4,Minor Arterial,06155MU00210 01NN**************,06155MU00210--1-----,1,South,2,C,0,8.87,56.13,52.30,47.70,1.5226,https://maps.roads.maryland.gov/itms_public/?s...,5520.0,5541.0,5502.0,4960.0,4951.0,5082.0,4910.0,5031.0,4992.0,5150,5910.0,5981.0,5942.0,5360.0,5301.0,5442.0,5250.0,5381.0,5342.0,5460,8.0,4287.0,705.0,10.0,130.0,10.0,NO ERROR,4215.520094,140.0,0.928571,0.071429,0.001553,0.832427,0.136893,0.001942,0.025243,0.001942,0.027184,2,8.87,56.13,52.30,47.70,"5,520","5,541","5,502","4,960","4,951","5,082","4,910","5,031","4,992","5,150","5,910","5,981","5,942","5,360","5,301","5,442","5,250","5,381","5,342","5,460",8.0,"4,287",10.0,705,130,10.0,140.0,1668.612144
84,85,B020046,2,Anne Arundel,3,Annapolis,Tyler Ave,MU,3220,,0.37,0.27,1.21,Tyler Ave-.10 Mi N Of Bay Ridge Ave,Bay Ridge Ave To Forest Dr,Urban,6,Minor Collector,02003MU03220 01NN**************,02003MU03220--1-----,1,North,3,F,0,8.19,54.66,56.11,43.89,3.8743,https://maps.roads.maryland.gov/itms_public/?s...,11692.0,11300.0,11211.0,11232.0,11680.0,11981.0,12212.0,11390.0,11291.0,11292,12512.0,12200.0,12111.0,12132.0,12500.0,12821.0,13072.0,12070.0,11971.0,12422,43.0,9228.0,1645.0,8.0,265.0,103.0,NO ERROR,4851.762379,368.0,0.720109,0.279891,0.003808,0.817216,0.145678,0.000708,0.023468,0.009121,0.032589,3,8.19,54.66,56.11,43.89,"11,692","11,300","11,211","11,232","11,680","11,981","12,212","11,390","11,291","11,292","12,512","12,200","12,111","12,132","12,500","12,821","13,072","12,070","11,971","12,422",43.0,"9,228",8.0,"1,645",265,103.0,368.0,1901.366299
188,189,B060072,6,Carroll,155,Westminster,Anchor St,MU,10,,0.05,0.00,0.08,Anchor St-between W. Main St & W. Green St (on...,W. Main St To W. Green St,Urban,4,Minor Arterial,06155MU00010 01SS**************,06155MU00010--1-----,1,South,1,C,0,7.93,100.00,0.00,100.00,0.0356,https://maps.roads.maryland.gov/itms_public/?s...,1140.0,1151.0,1142.0,1100.0,1101.0,1132.0,1170.0,1201.0,1192.0,1220,1220.0,1241.0,1232.0,1190.0,1181.0,1212.0,1250.0,1291.0,1282.0,1290,4.0,965.0,180.0,3.0,58.0,10.0,NO ERROR,469.215742,68.0,0.852941,0.147059,0.003279,0.790984,0.147541,0.002459,0.047541,0.008197,0.055738,1,7.93,100.00,0.00,100.00,"1,140","1,151","1,142","1,100","1,101","1,132","1,170","1,201","1,192","1,220","1,220","1,241","1,232","1,190","1,181","1,212","1,250","1,291","1,282","1,290",4.0,965,3.0,180,58,10.0,68.0,185.598061
261,262,B010016,1,Allegany,41,Cumberland,Bedford St,MU,270,,0.73,0.14,2

In [11]:
aadt_points_data.info()
aadt_points_data.columns
aadt_points_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 9 to 3008
Columns: 102 entries, X to TRUCK_AADT_ANNO
dtypes: float64(49), int64(11), object(42)
memory usage: 70.8+ KB


,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT_2010,AADT_2011,AADT_2012,AADT_2013,AADT_2014,AADT_2015,AADT_2016,AADT_2017,AADT_2018,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO
9,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https://maps.roads.maryland.gov/itms_public/?s...,1045.0,950.0,941.0,942.0,500.0,511.0,522.0,490.0,491.0,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.000000,0.000000,0.032520,0.831301,0.126016,0.004065,0.006098,0.000000,0.006098,2,9.41,100.00,100.00,0.00,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3
59,-8.470538e+06,4.661852e+06,60,B090013,9,Dorchester,19,Cambridge,Glasgow St,MU,520,,0.10,0.00,1.15,Glasgow St-.10 Mi S Of Hambrooks Blvd,Hambrooks Blvd To High St,Urban,5,Major Collector,09019MU00520 01SS**************,09019MU00520--1-----,1,North,2,F,0,9.01,61.69,47.89,52.11,1.0582,https://maps.roads.maryland.gov/itms_public/?s...,4931.0,4952.0,2510.0,2521.0,2512.0,2720.0,2771.0,2842.0,2520.0,2521,5281.0,5352.0,2710.0,2721.0,2692.0,2910.0,2971.0,3042.0,2700.0,2671,7.0,1871.0,492.0,8.0,127.0,16.0,NO ERROR,143.0,0.888112,0.111888,0.002777,0.742166,0.195161,0.003173,0.050377,0.006347,0.056724,2,9.01,61.69,47.89,52.11,"4,931","4,952","2,510","2,521","2,512","2,720","2,771","2,842","2,520","2,521","5,281","5,352","2,710","2,721","2,692","2,910","2,971","3,042","2,700","2,671",7.0,"1,871",8,492,127,16,143
79,-8.622809e+06,4.782320e+06,80,B100065,10,Frederick,57,Frederick,Waverley Dr,MU,1545,,0.20,0.00,0.78,Waverley Dr-.20 Mi N Of Us40,Us 40 To Shookstown Rd,Urban,6,Minor Collector,10057MU01545 01NN**************,10057MU01545--1-----,1,South,3,F,0,9.25,51.12,49.65,50.35,1.3928,https://maps.roads.maryland.gov/itms_public/?s...,5552.0,5810.0,5761.0,5772.0,4850.0,4981.0,5082.0,4930.0,4891.0,4892,5942.0,6280.0,6221.0,6232.0,5190.0,5331.0,5442.0,5230.0,5181.0,5382,43.0,3926.0,588.0,22.0,288.0,25.0,NO ERROR,313.0,0.920128,0.079872,0.008790,0.802535,0.120196,0.004497,0.058872,0.005110,0.063982,3,9.25,51.12,49.65,50.35,"5,552","5,810","5,761","5,772","4,850","4,981","5,082","4,930","4,891","4,892","5,942","6,280","6,221","6,232","5,190","5,331","5,442","5,230","5,181","5,382",43.0,"3,926",22,588,288,25,313
116,-8.519905e+06,4.718421e+06,117,B0767,2,Anne Arundel,3,Annapolis,Forest Dr,MU,1165,,0.56,0.36,1.21,Forest Dr-.10 Mi E Of Md393,Md 2 To Fairfax Rd,Urban,5,Major Collector,02003MU01165 01SS**************,02003MU01165--1-----,1,East,3,C,0,7

In [12]:
pedestrian_injury_data.info()
pedestrian_injury_data.columns
pedestrian_injury_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 268 entries, 0 to 349
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Jurisdiction     268 non-null    object 
 1   Value            268 non-null    float64
 2   Race/ ethnicity  268 non-null    object 
 3   Year             268 non-null    int64  
 4   Measure          268 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 12.6+ KB


,Jurisdiction,Value,Race/ ethnicity,Year,Measure
0,State,53.5,All races/ ethnicities (aggregated),2017,Pedestrian Injury
2,Anne Arundel,37.3,All races/ ethnicities (aggregated),2017,Pedestrian Injury
3,Baltimore City,183.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
4,Baltimore County,54.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
7,Carroll,11.9,All races/ ethnicities (aggregated),2017,Pedestrian Injury


In [43]:
# Identify columns representing years
points_year_columns = [col for col in aadt_points_data.columns if col.startswith('AADT_')]
points_year_columns_numeric = [col for col in points_year_columns if col.replace('AADT_', '').isdigit()]

In [44]:
# Melt the data to long format correctly
id_vars = [col for col in aadt_points_data.columns if col not in points_year_columns_numeric]

In [49]:
aadt_points_long = pd.melt(aadt_points_data, id_vars=id_vars, var_name='Year', value_name='AADT_points_year_value',
                    value_vars=points_year_columns_numeric)

In [50]:
# Correctly extracting the year from the 'Year' column
aadt_points_long['Year'] = aadt_points_long['Year'].str.replace('AADT_', '').astype(int)

# Print the first few rows of the resulting DataFrame to check the transformation
aadt_points_long.head()

,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Year,AADT_points_year_value
0,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.000000,0.000000,0.032520,0.831301,0.126016,0.004065,0.006098,0.000000,0.006098,2,9.41,100.00,100.00,0.00,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2010,1045.0
1,-8.470538e+06,4.661852e+06,60,B090013,9,Dorchester,19,Cambridge,Glasgow St,MU,520,,0.10,0.00,1.15,Glasgow St-.10 Mi S Of Hambrooks Blvd,Hambrooks Blvd To High St,Urban,5,Major Collector,09019MU00520 01SS**************,09019MU00520--1-----,1,North,2,F,0,9.01,61.69,47.89,52.11,1.0582,https://maps.roads.maryland.gov/itms_public/?s...,2521,5281.0,5352.0,2710.0,2721.0,2692.0,2910.0,2971.0,3042.0,2700.0,2671,7.0,1871.0,492.0,8.0,127.0,16.0,NO ERROR,143.0,0.888112,0.111888,0.002777,0.742166,0.195161,0.003173,0.050377,0.006347,0.056724,2,9.01,61.69,47.89,52.11,"4,931","4,952","2,510","2,521","2,512","2,720","2,771","2,842","2,520","2,521","5,281","5,352","2,710","2,721","2,692","2,910","2,971","3,042","2,700","2,671",7.0,"1,871",8,492,127,16,143,2010,4931.0
2,-8.622809e+06,4.782320e+06,80,B100065,10,Frederick,57,Frederick,Waverley Dr,MU,1545,,0.20,0.00,0.78,Waverley Dr-.20 Mi N Of Us40,Us 40 To Shookstown Rd,Urban,6,Minor Collector,10057MU01545 01NN**************,10057MU01545--1-----,1,South,3,F,0,9.25,51.12,49.65,50.35,1.3928,https://maps.roads.maryland.gov/itms_public/?s...,4892,5942.0,6280.0,6221.0,6232.0,5190.0,5331.0,5442.0,5230.0,5181.0,5382,43.0,3926.0,588.0,22.0,288.0,25.0,NO ERROR,313.0,0.920128,0.079872,0.008790,0.802535,0.120196,0.004497,0.058872,0.005110,0.063982,3,9.25,51.12,49.65,50.35,"5,552","5,810","5,761","5,772","4,850","4,981","5,082","4,930","4,891","4,892","5,942","6,280","6,221","6,232","5,190","5,331","5,442","5,230","5,181","5,382",43.0,"3,926",22,588,288,25,313,2010,5552.0
3,-8.519905e+06,4.718421e+06,117,B0767,2,Anne Arundel,3,Annapolis,Forest Dr,MU,1165,,0.56,0.36,1.21,Forest Dr-.10 Mi E Of Md393,Md 2 To Fairfax Rd,Urban,5,Major Collector,02003MU01165 01SS**************,02003MU01165--1-----,1,East,3,C,0,7.66,52.77,48.15,51.85,3.0280,https://maps.roads.maryland.gov/itms_public/?s...,9760,11590.0,11741.0,11652.0,10340.0,10211.0,10482.0,10230.0,10381.0,10282.0,10740,10.0,8724.0,857.0,8.0,132.0,29.0,NO ERROR,161.0,0

In [20]:
# Check unique values in 'Year' column before conversion
print(aadt_points_long['Year'].unique())

[2010 2011 2012 2013 2014 2015 2016 2017 2018]


In [21]:
try:
    aadt_points_long['Year'] = aadt_points_long['Year'].astype(int)
except ValueError as e:
    print(f"Error converting Year: {e}")

aadt_points_long.info()
aadt_points_long.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 95 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   X                             792 non-null    float64
 1   Y                             792 non-null    float64
 2   OBJECTID                      792 non-null    int64  
 3   LOCATION_ID                   792 non-null    object 
 4   COUNTY_ID                     792 non-null    int64  
 5   COUNTY_DESC                   792 non-null    object 
 6   MUN_SORT                      792 non-null    int64  
 7   MUNICIPALITY                  792 non-null    object 
 8   ROADNAME                      792 non-null    object 
 9   ID_PREFIX                     792 non-null    object 
 10  ID_RTE_NO                     792 non-null    int64  
 11  MP_SUFFIX                     792 non-null    object 
 12  ID_MP                         792 non-null    float64
 13  BEGIN

,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Year,AADT_year_value
0,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.000000,0.000000,0.032520,0.831301,0.126016,0.004065,0.006098,0.000000,0.006098,2,9.41,100.00,100.00,0.00,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2010,1045.0
1,-8.470538e+06,4.661852e+06,60,B090013,9,Dorchester,19,Cambridge,Glasgow St,MU,520,,0.10,0.00,1.15,Glasgow St-.10 Mi S Of Hambrooks Blvd,Hambrooks Blvd To High St,Urban,5,Major Collector,09019MU00520 01SS**************,09019MU00520--1-----,1,North,2,F,0,9.01,61.69,47.89,52.11,1.0582,https://maps.roads.maryland.gov/itms_public/?s...,2521,5281.0,5352.0,2710.0,2721.0,2692.0,2910.0,2971.0,3042.0,2700.0,2671,7.0,1871.0,492.0,8.0,127.0,16.0,NO ERROR,143.0,0.888112,0.111888,0.002777,0.742166,0.195161,0.003173,0.050377,0.006347,0.056724,2,9.01,61.69,47.89,52.11,"4,931","4,952","2,510","2,521","2,512","2,720","2,771","2,842","2,520","2,521","5,281","5,352","2,710","2,721","2,692","2,910","2,971","3,042","2,700","2,671",7.0,"1,871",8,492,127,16,143,2010,4931.0
2,-8.622809e+06,4.782320e+06,80,B100065,10,Frederick,57,Frederick,Waverley Dr,MU,1545,,0.20,0.00,0.78,Waverley Dr-.20 Mi N Of Us40,Us 40 To Shookstown Rd,Urban,6,Minor Collector,10057MU01545 01NN**************,10057MU01545--1-----,1,South,3,F,0,9.25,51.12,49.65,50.35,1.3928,https://maps.roads.maryland.gov/itms_public/?s...,4892,5942.0,6280.0,6221.0,6232.0,5190.0,5331.0,5442.0,5230.0,5181.0,5382,43.0,3926.0,588.0,22.0,288.0,25.0,NO ERROR,313.0,0.920128,0.079872,0.008790,0.802535,0.120196,0.004497,0.058872,0.005110,0.063982,3,9.25,51.12,49.65,50.35,"5,552","5,810","5,761","5,772","4,850","4,981","5,082","4,930","4,891","4,892","5,942","6,280","6,221","6,232","5,190","5,331","5,442","5,230","5,181","5,382",43.0,"3,926",22,588,288,25,313,2010,5552.0
3,-8.519905e+06,4.718421e+06,117,B0767,2,Anne Arundel,3,Annapolis,Forest Dr,MU,1165,,0.56,0.36,1.21,Forest Dr-.10 Mi E Of Md393,Md 2 To Fairfax Rd,Urban,5,Major Collector,02003MU01165 01SS**************,02003MU01165--1-----,1,East,3,C,0,7.66,52.77,48.15,51.85,3.0280,https://maps.roads.maryland.gov/itms_public/?s...,9760,11590.0,11741.0,11652.0,10340.0,10211.0,10482.0,10230.0,10381.0,10282.0,10740,10.0,8724.0,857.0,8.0,132.0,29.0,NO ERROR,161.0,0.819876

In [22]:
pedestrian_injury_data['Year'] = pedestrian_injury_data['Year'].astype(int)
pedestrian_injury_data.info()
pedestrian_injury_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 268 entries, 0 to 349
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Jurisdiction     268 non-null    object 
 1   Value            268 non-null    float64
 2   Race/ ethnicity  268 non-null    object 
 3   Year             268 non-null    int32  
 4   Measure          268 non-null    object 
dtypes: float64(1), int32(1), object(3)
memory usage: 11.5+ KB


,Jurisdiction,Value,Race/ ethnicity,Year,Measure
0,State,53.5,All races/ ethnicities (aggregated),2017,Pedestrian Injury
2,Anne Arundel,37.3,All races/ ethnicities (aggregated),2017,Pedestrian Injury
3,Baltimore City,183.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
4,Baltimore County,54.4,All races/ ethnicities (aggregated),2017,Pedestrian Injury
7,Carroll,11.9,All races/ ethnicities (aggregated),2017,Pedestrian Injury


In [23]:
pedestrian_injury_data['Year'].unique()

array([2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2018, 2019,
       2020, 2021, 2022])

In [24]:
traffic_stops_data.rename(columns={'year': 'Year'}, inplace=True)
traffic_stops_data['year'] = traffic_stops_data['Year'].astype(int)

traffic_stops_data.info()
traffic_stops_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Year                                  41 non-null     object
 1   quarter                               41 non-null     object
 2   quarter_and_year                      41 non-null     object
 3   medevac_missions                      41 non-null     object
 4   impaired_driving_arrests              41 non-null     object
 5   cross_border_criminal_investigations  41 non-null     object
 6   total_traffic_stops                   41 non-null     object
 7   year                                  41 non-null     int32 
dtypes: int32(1), object(7)
memory usage: 2.5+ KB


,Year,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops,year
0,2014,Q1,Q1 2014,376,1496,180,110734,2014
1,2014,Q2,Q2 2014,597,1630,104,141940,2014
2,2014,Q3,Q3 2014,681,1767,148,147807,2014
3,2014,Q4,Q4 2014,484,1562,108,119741,2014
4,2015,Q1,Q1 2015,345,1500,210,132779,2015


In [25]:
traffic_stops_data['Year'].unique()

array(['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '2024'], dtype=object)

In [26]:
# Merge the reshaped traffic data with pedestrian data
combined_data = pd.merge(aadt_points_long, pedestrian_injury_data, on='Year', how='inner')

In [27]:
# Now merge the combined data with traffic stops data
final_merged_data = pd.merge(combined_data, traffic_stops_data, left_on='Year', right_on='year', how='inner')

In [28]:
final_merged_data.head()

,X,Y,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Year_x,AADT_year_value,Jurisdiction,Value,Race/ ethnicity,Measure,Year_y,quarter,quarter_and_year,medevac_missions,impaired_driving_arrests,cross_border_criminal_investigations,total_traffic_stops,year
0,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.0,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.0,100.0,0.0,0.079,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.0,0.0,0.03252,0.831301,0.126016,0.004065,0.006098,0.0,0.006098,2,9.41,100.0,100.0,0.0,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2014,500.0,State,42.6,All races/ ethnicities (aggregated),Pedestrian Injury,2014,Q1,Q1 2014,376,1496,180,110734,2014
1,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.0,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.0,100.0,0.0,0.079,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.0,0.0,0.03252,0.831301,0.126016,0.004065,0.006098,0.0,0.006098,2,9.41,100.0,100.0,0.0,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2014,500.0,State,42.6,All races/ ethnicities (aggregated),Pedestrian Injury,2014,Q2,Q2 2014,597,1630,104,141940,2014
2,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.0,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.0,100.0,0.0,0.079,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.0,0.0,0.03252,0.831301,0.126016,0.004065,0.006098,0.0,0.006098,2,9.41,100.0,100.0,0.0,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2014,500.0,State,42.6,All races/ ethnicities (aggregated),Pedestrian Injury,2014,Q3,Q3 2014,681,1767,148,147807,2014
3,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.0,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virgin

In [29]:
final_merged_data['year'].unique()

array([2014, 2015, 2016, 2017, 2018])

In [52]:
lines_year_columns = [col for col in aadt_lines_data.columns if col.startswith('AADT_')]
lines_year_columns_numeric = [col for col in lines_year_columns if col.replace('AADT_', '').isdigit()]

# Melt the data to long format correctly
id_vars = [col for col in aadt_lines_data.columns if col not in lines_year_columns_numeric]

aadt_lines_long = pd.melt(aadt_lines_data, id_vars=id_vars, var_name='Year', value_name='AADT_lines_year_value',
                    value_vars=lines_year_columns_numeric)

# Correctly extracting the year from the 'Year' column
aadt_lines_long['Year'] = aadt_lines_long['Year'].str.replace('AADT_', '').astype(int)

# Print the first few rows of the resulting DataFrame to check the transformation
aadt_lines_long.head()

,OBJECTID,LOCATION_ID,COUNTY_ID,COUNTY_DESC,MUN_SORT,MUNICIPALITY,ROADNAME,ID_PREFIX,ID_RTE_NO,MP_SUFFIX,ID_MP,BEGIN_SECTION,END_SECTION,STATION_DESC,ROAD_SECTION,RURAL_URBAN,F_SYSTEM,F_SYSTEM_DESC,ROUTEID,ROUTEID_RH,MAIN_LINE,PEAK_HOUR_DIRECTION,NUM_LANES,COUNTED_FACTORED,STMP_SEQ,K_FACTOR,D_FACTOR,NORTH_EAST_SPLIT,SOUTH_WEST_SPLIT,AVMT,LINK,AADT,AAWDT_2010,AAWDT_2011,AAWDT_2012,AAWDT_2013,AAWDT_2014,AAWDT_2015,AAWDT_2016,AAWDT_2017,AAWDT_2018,AAWDT,MOTORCYCLE_AADT,CAR_AADT,LIGHT_TRUCK_AADT,BUS_AADT,SINGLE_UNIT_AADT,COMBINATION_UNIT_AADT,LOC_ERROR,Shape_Length0,TRUCK_AADT,SUTRUCK_AADT_PCT_TRUCK,CUTRUCK_AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL,CAR_AADT_PCT_OVERALL,LIGHT_TRUCK_AADT_PCT_OVERALL,BUS_AADT_PCT_OVERALL,SUTRUCK_AADT_PCT_OVERALL,CUTRUCK_AADT_PCT_OVERALL,TRUCK_AADT_PCT_OVERALL,NUM_LANES_ANNO,K_FACTOR_ANNO,D_FACTOR_ANNO,NORTH_EAST_SPLIT_ANNO,SOUTH_WEST_SPLIT_ANNO,AADT_2010_ANNO,AADT_2011_ANNO,AADT_2012_ANNO,AADT_2013_ANNO,AADT_2014_ANNO,AADT_2015_ANNO,AADT_2016_ANNO,AADT_2017_ANNO,AADT_2018_ANNO,AADT_ANNO,AAWDT_2010_ANNO,AAWDT_2011_ANNO,AAWDT_2012_ANNO,AAWDT_2013_ANNO,AAWDT_2014_ANNO,AAWDT_2015_ANNO,AAWDT_2016_ANNO,AAWDT_2017_ANNO,AAWDT_2018_ANNO,AAWDT_ANNO,MOTORCYCLE_AADT_ANNO,CAR_AADT_ANNO,BUS_AADT_ANNO,LIGHT_TRUCK_AADT_ANNO,SINGLE_UNIT_AADT_ANNO,COMBINATION_UNIT_AADT_ANNO,TRUCK_AADT_ANNO,Shape_Length,Year,AADT_lines_year_value
0,47,B060073,6,Carroll,155,Westminster,E Green St,MU,210,,0.23,0.17,0.98,Green St E&w-between Washington Rd & Bishop St,Washington Rd To Md 27,Urban,4,Minor Arterial,06155MU00210 01NN**************,06155MU00210--1-----,1,South,2,C,0,8.87,56.13,52.30,47.70,1.5226,https://maps.roads.maryland.gov/itms_public/?s...,5150,5910.0,5981.0,5942.0,5360.0,5301.0,5442.0,5250.0,5381.0,5342.0,5460,8.0,4287.0,705.0,10.0,130.0,10.0,NO ERROR,4215.520094,140.0,0.928571,0.071429,0.001553,0.832427,0.136893,0.001942,0.025243,0.001942,0.027184,2,8.87,56.13,52.30,47.70,"5,520","5,541","5,502","4,960","4,951","5,082","4,910","5,031","4,992","5,150","5,910","5,981","5,942","5,360","5,301","5,442","5,250","5,381","5,342","5,460",8.0,"4,287",10.0,705,130,10.0,140.0,1668.612144,2010,5520.0
1,85,B020046,2,Anne Arundel,3,Annapolis,Tyler Ave,MU,3220,,0.37,0.27,1.21,Tyler Ave-.10 Mi N Of Bay Ridge Ave,Bay Ridge Ave To Forest Dr,Urban,6,Minor Collector,02003MU03220 01NN**************,02003MU03220--1-----,1,North,3,F,0,8.19,54.66,56.11,43.89,3.8743,https://maps.roads.maryland.gov/itms_public/?s...,11292,12512.0,12200.0,12111.0,12132.0,12500.0,12821.0,13072.0,12070.0,11971.0,12422,43.0,9228.0,1645.0,8.0,265.0,103.0,NO ERROR,4851.762379,368.0,0.720109,0.279891,0.003808,0.817216,0.145678,0.000708,0.023468,0.009121,0.032589,3,8.19,54.66,56.11,43.89,"11,692","11,300","11,211","11,232","11,680","11,981","12,212","11,390","11,291","11,292","12,512","12,200","12,111","12,132","12,500","12,821","13,072","12,070","11,971","12,422",43.0,"9,228",8.0,"1,645",265,103.0,368.0,1901.366299,2010,11692.0
2,189,B060072,6,Carroll,155,Westminster,Anchor St,MU,10,,0.05,0.00,0.08,Anchor St-between W. Main St & W. Green St (on...,W. Main St To W. Green St,Urban,4,Minor Arterial,06155MU00010 01SS**************,06155MU00010--1-----,1,South,1,C,0,7.93,100.00,0.00,100.00,0.0356,https://maps.roads.maryland.gov/itms_public/?s...,1220,1220.0,1241.0,1232.0,1190.0,1181.0,1212.0,1250.0,1291.0,1282.0,1290,4.0,965.0,180.0,3.0,58.0,10.0,NO ERROR,469.215742,68.0,0.852941,0.147059,0.003279,0.790984,0.147541,0.002459,0.047541,0.008197,0.055738,1,7.93,100.00,0.00,100.00,"1,140","1,151","1,142","1,100","1,101","1,132","1,170","1,201","1,192","1,220","1,220","1,241","1,232","1,190","1,181","1,212","1,250","1,291","1,282","1,290",4.0,965,3.0,180,58,10.0,68.0,185.598061,2010,1140.0
3,262,B010016,1,Allegany,41,Cumberland,Bedford St,MU,270,,0.73,0.14,2.23,Bedford St-.70 Mi S Of Md807 (one Way),Md 807 (back) To Mechanic St,Urban,4,Minor Arterial,01041MU00270 01SS**************,01041MU00270--1-----,1,South,2,C,0,8.25,100.00,0.00,100.00,1.4952,https://maps.roads.maryland.gov/itms_

In [54]:
print(aadt_lines_long['Year'].unique())

[2010 2011 2012 2013 2014 2015 2016 2017 2018]


In [55]:
try:
    aadt_lines_long['Year'] = aadt_lines_long['Year'].astype(int)
except ValueError as e:
    print(f"Error converting Year: {e}")

In [69]:
# Assuming aadt_lines_data has been reshaped to aadt_lines_long
traffic_data = pd.merge(aadt_points_long, aadt_lines_long, on=['Year', 'LOCATION_ID'], how='inner')
traffic_data.head()

,X,Y,OBJECTID_x,LOCATION_ID,COUNTY_ID_x,COUNTY_DESC_x,MUN_SORT_x,MUNICIPALITY_x,ROADNAME_x,ID_PREFIX_x,ID_RTE_NO_x,MP_SUFFIX_x,ID_MP_x,BEGIN_SECTION_x,END_SECTION_x,STATION_DESC_x,ROAD_SECTION_x,RURAL_URBAN_x,F_SYSTEM_x,F_SYSTEM_DESC_x,ROUTEID_x,ROUTEID_RH_x,MAIN_LINE_x,PEAK_HOUR_DIRECTION_x,NUM_LANES_x,COUNTED_FACTORED_x,STMP_SEQ_x,K_FACTOR_x,D_FACTOR_x,NORTH_EAST_SPLIT_x,SOUTH_WEST_SPLIT_x,AVMT_x,LINK_x,AADT_x,AAWDT_2010_x,AAWDT_2011_x,AAWDT_2012_x,AAWDT_2013_x,AAWDT_2014_x,AAWDT_2015_x,AAWDT_2016_x,AAWDT_2017_x,AAWDT_2018_x,AAWDT_x,MOTORCYCLE_AADT_x,CAR_AADT_x,LIGHT_TRUCK_AADT_x,BUS_AADT_x,SINGLE_UNIT_AADT_x,COMBINATION_UNIT_AADT_x,LOC_ERROR_x,TRUCK_AADT_x,SUTRUCK_AADT_PCT_TRUCK_x,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL_x,CAR_AADT_PCT_OVERALL_x,LIGHT_TRUCK_AADT_PCT_OVERALL_x,BUS_AADT_PCT_OVERALL_x,SUTRUCK_AADT_PCT_OVERALL_x,CUTRUCK_AADT_PCT_OVERALL_x,TRUCK_AADT_PCT_OVERALL_x,NUM_LANES_ANNO_x,K_FACTOR_ANNO_x,D_FACTOR_ANNO_x,NORTH_EAST_SPLIT_ANNO_x,SOUTH_WEST_SPLIT_ANNO_x,AADT_2010_ANNO_x,AADT_2011_ANNO_x,AADT_2012_ANNO_x,AADT_2013_ANNO_x,AADT_2014_ANNO_x,AADT_2015_ANNO_x,AADT_2016_ANNO_x,AADT_2017_ANNO_x,AADT_2018_ANNO_x,AADT_ANNO_x,AAWDT_2010_ANNO_x,AAWDT_2011_ANNO_x,AAWDT_2012_ANNO_x,AAWDT_2013_ANNO_x,AAWDT_2014_ANNO_x,AAWDT_2015_ANNO_x,AAWDT_2016_ANNO_x,AAWDT_2017_ANNO_x,AAWDT_2018_ANNO_x,AAWDT_ANNO_x,MOTORCYCLE_AADT_ANNO_x,CAR_AADT_ANNO_x,BUS_AADT_ANNO_x,LIGHT_TRUCK_AADT_ANNO_x,SINGLE_UNIT_AADT_ANNO_x,COMBINATION_UNIT_AADT_ANNO_x,TRUCK_AADT_ANNO_x,Year,AADT_points_year_value,OBJECTID_y,COUNTY_ID_y,COUNTY_DESC_y,MUN_SORT_y,MUNICIPALITY_y,ROADNAME_y,ID_PREFIX_y,ID_RTE_NO_y,MP_SUFFIX_y,ID_MP_y,BEGIN_SECTION_y,END_SECTION_y,STATION_DESC_y,ROAD_SECTION_y,RURAL_URBAN_y,F_SYSTEM_y,F_SYSTEM_DESC_y,ROUTEID_y,ROUTEID_RH_y,MAIN_LINE_y,PEAK_HOUR_DIRECTION_y,NUM_LANES_y,COUNTED_FACTORED_y,STMP_SEQ_y,K_FACTOR_y,D_FACTOR_y,NORTH_EAST_SPLIT_y,SOUTH_WEST_SPLIT_y,AVMT_y,LINK_y,AADT_y,AAWDT_2010_y,AAWDT_2011_y,AAWDT_2012_y,AAWDT_2013_y,AAWDT_2014_y,AAWDT_2015_y,AAWDT_2016_y,AAWDT_2017_y,AAWDT_2018_y,AAWDT_y,MOTORCYCLE_AADT_y,CAR_AADT_y,LIGHT_TRUCK_AADT_y,BUS_AADT_y,SINGLE_UNIT_AADT_y,COMBINATION_UNIT_AADT_y,LOC_ERROR_y,Shape_Length0,TRUCK_AADT_y,SUTRUCK_AADT_PCT_TRUCK_y,CUTRUCK_AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL_y,CAR_AADT_PCT_OVERALL_y,LIGHT_TRUCK_AADT_PCT_OVERALL_y,BUS_AADT_PCT_OVERALL_y,SUTRUCK_AADT_PCT_OVERALL_y,CUTRUCK_AADT_PCT_OVERALL_y,TRUCK_AADT_PCT_OVERALL_y,NUM_LANES_ANNO_y,K_FACTOR_ANNO_y,D_FACTOR_ANNO_y,NORTH_EAST_SPLIT_ANNO_y,SOUTH_WEST_SPLIT_ANNO_y,AADT_2010_ANNO_y,AADT_2011_ANNO_y,AADT_2012_ANNO_y,AADT_2013_ANNO_y,AADT_2014_ANNO_y,AADT_2015_ANNO_y,AADT_2016_ANNO_y,AADT_2017_ANNO_y,AADT_2018_ANNO_y,AADT_ANNO_y,AAWDT_2010_ANNO_y,AAWDT_2011_ANNO_y,AAWDT_2012_ANNO_y,AAWDT_2013_ANNO_y,AAWDT_2014_ANNO_y,AAWDT_2015_ANNO_y,AAWDT_2016_ANNO_y,AAWDT_2017_ANNO_y,AAWDT_2018_ANNO_y,AAWDT_ANNO_y,MOTORCYCLE_AADT_ANNO_y,CAR_AADT_ANNO_y,BUS_AADT_ANNO_y,LIGHT_TRUCK_AADT_ANNO_y,SINGLE_UNIT_AADT_ANNO_y,COMBINATION_UNIT_AADT_ANNO_y,TRUCK_AADT_ANNO_y,Shape_Length,AADT_lines_year_value
0,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https://maps.roads.maryland.gov/itms_public/?s...,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.000000,0.000000,0.032520,0.831301,0.126016,0.004065,0.006098,0.000000,0.006098,2,9.41,100.00,100.00,0.00,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,2010,1045.0,752,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https:/

In [102]:
merged_traffic_data = pd.merge(aadt_points_data, aadt_lines_data, on='LOCATION_ID', how='inner')

merged_traffic_data.head()

,X,Y,OBJECTID_x,LOCATION_ID,COUNTY_ID_x,COUNTY_DESC_x,MUN_SORT_x,MUNICIPALITY_x,ROADNAME_x,ID_PREFIX_x,ID_RTE_NO_x,MP_SUFFIX_x,ID_MP_x,BEGIN_SECTION_x,END_SECTION_x,STATION_DESC_x,ROAD_SECTION_x,RURAL_URBAN_x,F_SYSTEM_x,F_SYSTEM_DESC_x,ROUTEID_x,ROUTEID_RH_x,MAIN_LINE_x,PEAK_HOUR_DIRECTION_x,NUM_LANES_x,COUNTED_FACTORED_x,STMP_SEQ_x,K_FACTOR_x,D_FACTOR_x,NORTH_EAST_SPLIT_x,SOUTH_WEST_SPLIT_x,AVMT_x,LINK_x,AADT_2010_x,AADT_2011_x,AADT_2012_x,AADT_2013_x,AADT_2014_x,AADT_2015_x,AADT_2016_x,AADT_2017_x,AADT_2018_x,AADT_x,AAWDT_2010_x,AAWDT_2011_x,AAWDT_2012_x,AAWDT_2013_x,AAWDT_2014_x,AAWDT_2015_x,AAWDT_2016_x,AAWDT_2017_x,AAWDT_2018_x,AAWDT_x,MOTORCYCLE_AADT_x,CAR_AADT_x,LIGHT_TRUCK_AADT_x,BUS_AADT_x,SINGLE_UNIT_AADT_x,COMBINATION_UNIT_AADT_x,LOC_ERROR_x,TRUCK_AADT_x,SUTRUCK_AADT_PCT_TRUCK_x,CUTRUCK__AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL_x,CAR_AADT_PCT_OVERALL_x,LIGHT_TRUCK_AADT_PCT_OVERALL_x,BUS_AADT_PCT_OVERALL_x,SUTRUCK_AADT_PCT_OVERALL_x,CUTRUCK_AADT_PCT_OVERALL_x,TRUCK_AADT_PCT_OVERALL_x,NUM_LANES_ANNO_x,K_FACTOR_ANNO_x,D_FACTOR_ANNO_x,NORTH_EAST_SPLIT_ANNO_x,SOUTH_WEST_SPLIT_ANNO_x,AADT_2010_ANNO_x,AADT_2011_ANNO_x,AADT_2012_ANNO_x,AADT_2013_ANNO_x,AADT_2014_ANNO_x,AADT_2015_ANNO_x,AADT_2016_ANNO_x,AADT_2017_ANNO_x,AADT_2018_ANNO_x,AADT_ANNO_x,AAWDT_2010_ANNO_x,AAWDT_2011_ANNO_x,AAWDT_2012_ANNO_x,AAWDT_2013_ANNO_x,AAWDT_2014_ANNO_x,AAWDT_2015_ANNO_x,AAWDT_2016_ANNO_x,AAWDT_2017_ANNO_x,AAWDT_2018_ANNO_x,AAWDT_ANNO_x,MOTORCYCLE_AADT_ANNO_x,CAR_AADT_ANNO_x,BUS_AADT_ANNO_x,LIGHT_TRUCK_AADT_ANNO_x,SINGLE_UNIT_AADT_ANNO_x,COMBINATION_UNIT_AADT_ANNO_x,TRUCK_AADT_ANNO_x,OBJECTID_y,COUNTY_ID_y,COUNTY_DESC_y,MUN_SORT_y,MUNICIPALITY_y,ROADNAME_y,ID_PREFIX_y,ID_RTE_NO_y,MP_SUFFIX_y,ID_MP_y,BEGIN_SECTION_y,END_SECTION_y,STATION_DESC_y,ROAD_SECTION_y,RURAL_URBAN_y,F_SYSTEM_y,F_SYSTEM_DESC_y,ROUTEID_y,ROUTEID_RH_y,MAIN_LINE_y,PEAK_HOUR_DIRECTION_y,NUM_LANES_y,COUNTED_FACTORED_y,STMP_SEQ_y,K_FACTOR_y,D_FACTOR_y,NORTH_EAST_SPLIT_y,SOUTH_WEST_SPLIT_y,AVMT_y,LINK_y,AADT_2010_y,AADT_2011_y,AADT_2012_y,AADT_2013_y,AADT_2014_y,AADT_2015_y,AADT_2016_y,AADT_2017_y,AADT_2018_y,AADT_y,AAWDT_2010_y,AAWDT_2011_y,AAWDT_2012_y,AAWDT_2013_y,AAWDT_2014_y,AAWDT_2015_y,AAWDT_2016_y,AAWDT_2017_y,AAWDT_2018_y,AAWDT_y,MOTORCYCLE_AADT_y,CAR_AADT_y,LIGHT_TRUCK_AADT_y,BUS_AADT_y,SINGLE_UNIT_AADT_y,COMBINATION_UNIT_AADT_y,LOC_ERROR_y,Shape_Length0,TRUCK_AADT_y,SUTRUCK_AADT_PCT_TRUCK_y,CUTRUCK_AADT_PCT_TRUCK,MCYCLE_AADT_PCT_OVERALL_y,CAR_AADT_PCT_OVERALL_y,LIGHT_TRUCK_AADT_PCT_OVERALL_y,BUS_AADT_PCT_OVERALL_y,SUTRUCK_AADT_PCT_OVERALL_y,CUTRUCK_AADT_PCT_OVERALL_y,TRUCK_AADT_PCT_OVERALL_y,NUM_LANES_ANNO_y,K_FACTOR_ANNO_y,D_FACTOR_ANNO_y,NORTH_EAST_SPLIT_ANNO_y,SOUTH_WEST_SPLIT_ANNO_y,AADT_2010_ANNO_y,AADT_2011_ANNO_y,AADT_2012_ANNO_y,AADT_2013_ANNO_y,AADT_2014_ANNO_y,AADT_2015_ANNO_y,AADT_2016_ANNO_y,AADT_2017_ANNO_y,AADT_2018_ANNO_y,AADT_ANNO_y,AAWDT_2010_ANNO_y,AAWDT_2011_ANNO_y,AAWDT_2012_ANNO_y,AAWDT_2013_ANNO_y,AAWDT_2014_ANNO_y,AAWDT_2015_ANNO_y,AAWDT_2016_ANNO_y,AAWDT_2017_ANNO_y,AAWDT_2018_ANNO_y,AAWDT_ANNO_y,MOTORCYCLE_AADT_ANNO_y,CAR_AADT_ANNO_y,BUS_AADT_ANNO_y,LIGHT_TRUCK_AADT_ANNO_y,SINGLE_UNIT_AADT_ANNO_y,COMBINATION_UNIT_AADT_ANNO_y,TRUCK_AADT_ANNO_y,Shape_Length
0,-8.767689e+06,4.813219e+06,10,B010029,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.44,E Second St-.between Arch St & Grand Ave (one ...,Virginia Ave To Memorial Ave,Urban,6,Minor Collector,01041MU03210 01EE**************,01041MU03210--1-----,1,East,2,F,0,9.41,100.00,100.00,0.00,0.0790,https://maps.roads.maryland.gov/itms_public/?s...,1045.0,950.0,941.0,942.0,500.0,511.0,522.0,490.0,491.0,492,1105.0,1030.0,1021.0,1022.0,540.0,551.0,562.0,520.0,521.0,542,16.0,409.0,62.0,2.0,3.0,0.0,NO ERROR,3.0,1.000000,0.000000,0.032520,0.831301,0.126016,0.004065,0.006098,0.000000,0.006098,2,9.41,100.00,100.00,0.00,"1,045",950,941,942,500,511,522,490,491,492,"1,105","1,030","1,021","1,022",540,551,562,520,521,542,16.0,409,2,62,3,0,3,752,1,Allegany,41,Cumberland,E Second St,MU,3210,,0.07,0.00,0.4

In [103]:
annual_traffic = merged_traffic_data.groupby('YEAR').agg({'AADT': 'sum'})

KeyError: 'YEAR'

In [104]:
# Merging data examples
# Assuming common columns 'Date' and 'Location' for simplicity
combined_data = pd.merge(aadt_points_data, aadt_lines_data, on=['Location'], how='inner')
combined_data = pd.merge(combined_data, pedestrian_injury_data, on=['Date', 'Location'], how='inner')

KeyError: 'Location'

In [ ]:
print("Columns in aadt_points_data:", aadt_points_data.columns)
print("Columns in aadt_lines_data:", aadt_lines_data.columns)
print("Columns in pedestrian_injury_data:", pedestrian_injury_data.columns)


In [ ]:
# Statistical summary and correlations
print(combined_data.describe())
sns.heatmap(combined_data.corr(), annot=True)
plt.show()


In [ ]:
# More complex analyses like regression, clustering, etc.
from sklearn.linear_model import LinearRegression

# Example: Linear regression to predict AADT from other factors
model = LinearRegression()
model.fit(combined_data[['Num_Lanes']], combined_data['AADT'])
predictions = model.predict(combined_data[['Num_Lanes']])


In [ ]:
# Geographic visualization of traffic data
map = folium.Map(location=[38.9072, -76.8569], zoom_start=10)
for idx, row in combined_data.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']], radius=5, popup=str(row['AADT']), color='red').add_to(map)
map.save('Traffic_Map.html')


In [ ]:
# Summarize key findings and provide recommendations based on the analysis
print("Traffic volumes are highest on roads with X characteristic, suggesting Y policy interventions.")
